In [ ]:
import requests
import json
import pandas as pd
import os
import re
from dotenv import load_dotenv
from IPython.display import display, HTML # display와 HTML을 임포트

# .env 파일에서 환경변수 로드
load_dotenv()

# 네이버 API 헤더 설정
headers = {
    'X-Naver-Client-Id': os.getenv('NAVER_CLIENT_ID'),
    'X-Naver-Client-Secret': os.getenv('NAVER_CLIENT_SECRET')
}

def search_books(query):
    """
    네이버 책 검색 API를 호출하여 책 정보를 가져오는 함수
    """
    if not headers['X-Naver-Client-Id'] or not headers['X-Naver-Client-Secret']:
        print("오류: .env 파일에 NAVER_CLIENT_ID와 NAVER_CLIENT_SECRET을 설정해주세요.")
        return []
    
    payload = {
        'query': query,
        'display': 50,
        'sort': 'sim'
    }
    
    url = 'https://openapi.naver.com/v1/search/book.json'
    
    try:
        res = requests.get(url, params=payload, headers=headers)
        res.raise_for_status()
        return res.json()['items']
    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
        return []
    except KeyError as e:
        print(f"응답 데이터 파싱 중 오류 발생: {e}")
        return []

def save_books_to_json(books_data, filename='data/books.json'):
    """책 데이터를 JSON 파일로 저장하는 함수"""
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(books_data, f, ensure_ascii=False, indent=2)
        print(f"책 데이터가 {filename}에 저장되었습니다.")
    except Exception as e:
        print(f"파일 저장 중 오류 발생: {e}")

def load_books_from_json(filename='data/books.json'):
    """JSON 파일에서 책 데이터를 로드하는 함수"""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"{filename} 파일을 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"파일 로드 중 오류 발생: {e}")
        return []

def books_to_dataframe(books_data):
    """책 데이터를 Pandas DataFrame으로 변환하는 함수"""
    if not books_data:
        print("변환할 데이터가 없습니다.")
        return pd.DataFrame()
    
    def clean_html_tags(text):
        if isinstance(text, str):
            return re.sub('<.*?>', '', text)
        return text
    
    df = pd.DataFrame(books_data)
    
    html_columns = ['title', 'author', 'publisher', 'description']
    for col in html_columns:
        if col in df.columns:
            df[col] = df[col].apply(clean_html_tags)
    
    if 'discount' in df.columns:
        df['discount'] = df['discount'].astype(str).str.replace(',', '')
        df['discount'] = pd.to_numeric(df['discount'], errors='coerce')
        df['discount'] = df['discount'].fillna(0).astype(int)
    
    return df

def print_books_list(df):
    """책 목록을 출력하는 함수 (Jupyter에서는 DataFrame 자체를 출력하는 것이 일반적)"""
    if df.empty:
        print("출력할 책 정보가 없습니다.")
        return
    
    print("=" * 100)
    print("검색된 책 목록")
    print("=" * 100)
    
    for idx, row in df.iterrows():
        print(f"제목: {row.get('title', 'N/A')}")
        print(f"저자: {row.get('author', 'N/A')}")
        print(f"출판사: {row.get('publisher', 'N/A')}")
        print(f"출간일: {row.get('pubdate', 'N/A')}")
        print(f"할인가격: {row.get('discount', 'N/A')}원")
        print(f"ISBN: {row.get('isbn', 'N/A')}")
        print("-" * 80)

def filter_books_by_price(df, min_price=20000):
    """가격 조건으로 책을 필터링하는 함수"""
    if df.empty:
        print("필터링할 데이터가 없습니다.")
        return pd.DataFrame()
    
    filtered_df = df[df['discount'] >= min_price].copy()
    
    if filtered_df.empty:
        return pd.DataFrame()
    
    columns_to_show = ['title', 'author', 'discount', 'publisher', 'pubdate']
    available_columns = [col for col in filtered_df.columns if col in columns_to_show]
    filtered_df = filtered_df[available_columns]
    
    return filtered_df.sort_values('discount', ascending=False).reset_index(drop=True)

def filter_books_by_publisher(df, publisher_name="인피니티북스"):
    """출판사로 책을 필터링하는 함수"""
    if df.empty:
        print("필터링할 데이터가 없습니다.")
        return pd.DataFrame()
    
    filtered_df = df[df['publisher'].str.contains(publisher_name, na=False, case=False)].copy()
    
    if filtered_df.empty:
        return pd.DataFrame()
    
    columns_to_exclude = ['image', 'description']
    columns_to_show = [col for col in filtered_df.columns if col not in columns_to_exclude]
    return filtered_df[columns_to_show].reset_index(drop=True)

def display_aggregated_results(df):
    if df.empty:
        print("집계할 데이터가 없습니다.")
        return
    
    print("\n" + "=" * 80)
    print("📊 집계 결과 표 데이터 출력")
    print("=" * 80)
    
    # 3번 항목: 전체 책 목록 
    print("\n[3번 항목] 전체 책 목록")
    display_df_for_all_books = df[['title', 'link', 'image', 'author', 'discount', 'publisher', 'pubdate', 'isbn']].copy()
    display(display_df_for_all_books) 
    
    # 4번 항목: 2만원 이상 책 목록 
    print("\n[4번 항목] 2만원 이상 책 목록")
    expensive_books = df[df['discount'] >= 20000].copy() 
    if not expensive_books.empty:
        # 요청하신 컬럼만 선택하고, DataFrame으로 출력합니다.
        display_df_for_expensive_books = expensive_books[['title', 'author', 'discount', 'publisher', 'pubdate']].copy()
        display(display_df_for_expensive_books.sort_values('discount', ascending=False).reset_index(drop=True))
    else:
        print("2만원 이상인 책이 없습니다.")
    
    # 6번 항목: 인피니티북스 책 목록
    print("\n[6번 항목] 인피니티북스 출간 책 목록") # 제목 변경
    infinity_books = df[df['publisher'].str.contains('인피니티북스', na=False)].copy() 
    if not infinity_books.empty:

        columns_for_infinity_books = ['title', 'link', 'image', 'author', 'discount', 'publisher', 'pubdate', 'isbn']
        
        available_columns_for_infinity = [col for col in columns_for_infinity_books if col in infinity_books.columns]
        
        infinity_books_display = infinity_books[available_columns_for_infinity].sort_values('discount', ascending=False).reset_index(drop=True)
        display(infinity_books_display)
    else:
        print("인피니티북스에서 출간한 책이 없습니다.")

def main():
    search_query = "파이썬"
    print(f"'{search_query}' 검색 시작...")
    
    books_data = search_books(search_query)
    if not books_data:
        print("검색된 책이 없습니다.")
        return
    
    print(f"총 {len(books_data)}권의 책을 찾았습니다.")
    
    save_books_to_json(books_data)
    df = books_to_dataframe(books_data)
    
    # 기본 정보 출력
    print("\n[기본 정보]")
    print(f"총 {len(df)}권, 컬럼 수: {len(df.columns)}")
    print(f"컬럼 목록: {list(df.columns)}")
    
    # 집계 결과 출력
    display_aggregated_results(df)
    
    return df

if __name__ == "__main__":
    if not os.getenv('NAVER_CLIENT_ID') or not os.getenv('NAVER_CLIENT_SECRET'):
        print("⚠️ .env 파일에 API 키를 설정해주세요")
    else:
        print("✅ API 키가 정상적으로 로드되었습니다.")
        df = main()